In [1]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter
from itertools import chain,combinations

In [1]:
# https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6

from collect_data import load_cache
from tokenizer import load_tokens

tokens = load_tokens('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/tokens/text')
data = load_cache('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/snapshots')

loading tokens...:   0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

In [9]:
data.timestamp_ms.unique().shape

(10248338,)

In [46]:
hostname= "http://idea-vm-elasticsearch:9200"
index_name= "blockchain-interoperability-attacks"
mainquery= {
    "sort" : [
        { "created_at" : "asc" }
    ],
    "query": {
        "bool": {
            "filter": [
                {
                    "range": {
                        "created_at": {
                            "gte": "2022-11-07T00:00:00",
                            "lt": "now/s",
                            "format": "YYYY-MM-DD'T'HH:mm:ss",
                        }
                    },
                },
                {
                    "bool": {
                        "must_not": {
                            "exists": {
                                "field": "retweeted_status.id"
                            }
                        }
                    }
                }
            ]
        }
    }
}

In [31]:
data.columns

Index(['in_reply_to_status_id', 'created_at', 'truncated', 'timestamp_ms',
       'reply_count', 'retweet_count', 'entities.hashtags', 'favorite_count',
       'id', 'text', 'lang', 'quote_count', 'user.description', 'user.id',
       'extended_tweet.entities.hashtags', 'extended_tweet.full_text',
       'whole_text'],
      dtype='object')

In [21]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
        hosts=[hostname],
        verify_certs=False,
        # timeout=config.elasticsearch_timeout_secs
    )    

In [47]:
res = es.search(
    body = mainquery,
    index = index_name,
    size=10,
)

In [49]:
pd.json_normalize(res['hits']['hits'])

,_index,_type,_id,_score,sort,_source.created_at,_source.id,_source.id_str,_source.text,_source.display_text_range,...,_source.quoted_status.entities.user_mentions,_source.quoted_status.entities.symbols,_source.quoted_status.favorited,_source.quoted_status.retweeted,_source.quoted_status.possibly_sensitive,_source.quoted_status.filter_level,_source.quoted_status.lang,_source.quoted_status_permalink.url,_source.quoted_status_permalink.expanded,_source.quoted_status_permalink.display
0,blockchain-interoperability-attacks,_doc,1477801494212718592,None,[1641170307000],Mon Jan 03 00:38:27 +0000 2022,1477801494212718592,1477801494212718592,I made too much food again ! 😂 I have to do be...,"[0, 74]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,blockchain-interoperability-attacks,_doc,1477802142698246152,None,[1641170461000],Mon Jan 03 00:41:01 +0000 2022,1477802142698246152,1477802142698246152,Crypto Mories are lil mental genius’s There en...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,blockchain-interoperability-attacks,_doc,1477808035959644168,None,[1641171866000],Mon Jan 03 01:04:26 +0000 2022,1477808035959644168,1477808035959644168,Don’t Be A Statistic’ #IDO #HTML \n#trendingki...,"[0, 140]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,blockchain-interoperability-attacks,_doc,1477822518635364352,None,[1641175319000],Mon Jan 03 02:01:59 +0000 2022,1477822518635364352,1477822518635364352,always making my eyes roll back 🥵 https://t.co...,"[0, 33]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,blockchain-interoperability-attacks,_doc,1477842735247904768,None,[1641180139000],Mon Jan 03 03:22:19 +0000 2022,1477842735247904768,1477842735247904768,90’s vs Now📸’A Time Capsule’\n#trendingkiiara ...,"[0, 140]",...,"[{'screen_name': 'KIIARA', 'name': 'kiiara', '...",[],False,False,False,low,en,https://t.co/i2flz2QNYJ,https://twitter.com/Enter_Entendre/status/1477...,twitter.com/Enter_Entendre…
5,blockchain-interoperability-attacks,_doc,1477854072908697604,None,[1641182842000],Mon Jan 03 04:07:22 +0000 2022,1477854072908697604,1477854072908697604,Matt Damon doing a crypto ad. Jesus Christ doe...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,blockchain-interoperability-attacks,_doc,1477854571586146304,None,[1641182961000],Mon Jan 03 04:09:21 +0000 2022,1477854571586146304,1477854571586146304,If you’re not bullish on the #crypto markets t...,"[0, 140]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,blockchain-interoperability-attacks,_doc,1477861957340332032,None,[1641184722000],Mon Jan 03 04:38:42 +0000 2022,1477861957340332032,1477861957340332032,Happy New Year 2022!\n\nThese are the latest i...,"[0, 140]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,blockchain-interoperability-attacks,_doc,1477875739601035268,None,[1641188008000],Mon Jan 03 05:33:28 +0000 2022,1477875739601035268,1477875739601035268,#crypto #Nft #discord giveaway for #hexicans a...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,blockchain-interoperability-attacks,_doc,1477878451231236097,None,[1641188655000],Mon Jan 03 05:44:15 +0000 2022,1477878451231236097,1477878451231236097,📹something about dancing goofy in front of sec...,"[0, 70]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import pandas as pd
ts = pd.to_datetime(data.timestamp_ms.apply(float),unit='ms')
ts.min(), ts.max(), 

(Timestamp('2022-10-26 20:36:41.550000'),
 Timestamp('2022-11-16 01:04:52.597000'))